In [14]:
import pciSeq
import scipy.io
from scipy.sparse import coo_matrix
import pandas as pd
import scanpy as sc

#INPUT: molecules.csv, singlecell.h5ad labels.mat
#OUTPUT: assignments.csv
#From config:
segmentation_method = 'imagej'
molecules = "C:/Users/Habib/Projects/HMGU/tx_project/heart/raw_data/spots_PCW4.5_1.csv"
sc_data = "C:/Users/Habib/Projects/HMGU/tx_project/heart/raw_data/heart_sc.h5ad" 
opts = {'exclude_genes': ['TCIM']}

#Read and format molecules, single cell data, and labels
spots = pd.read_csv(molecules)
spots.columns = ['Gene', 'x', 'y']

adata = sc.read_h5ad(sc_data)
scdata = adata.X
scdata  = pd.DataFrame(scdata.transpose())
scdata.columns = adata.obs['celltype']
scdata.index = adata.var_names

label = scipy.io.loadmat('data/label_{}'.format(segmentation_method))['label']
coo = coo_matrix(label)

#Run through pciSeq
pciSeq.attach_to_log()
cellData, geneData = pciSeq.fit(spots, coo, scdata, opts)

#Save in correct format
assignments = geneData[ ["Gene", "x", "y", "neighbour"] ]
assignments.columns = ["gene", "x", "y", "cell"]
assignments.to_csv("data/assignments_{}_pciseq.csv".format(segmentation_method), index = False)

[2022-07-19 14:16:57] INFO    (app.py:151)  exclude_genes is set to ['TCIM']
[2022-07-19 14:16:57] INFO    (app.py: 79)  Preprocessing data
[2022-07-19 14:17:00] INFO    (spot_labels.py: 64)  Number of spots passed-in: 55240
[2022-07-19 14:17:02] INFO    (spot_labels.py: 65)  Number of segmented cells: 15451
[2022-07-19 14:17:02] INFO    (spot_labels.py: 66)  Segmentation array implies that image has width: 11194px and height: 11722px
[2022-07-19 14:20:52] INFO    (datatypes.py:371)  Single cell data passed-in have 15323 genes and 3777 cells
[2022-07-19 14:20:52] INFO    (datatypes.py:373)  Single cell data: Keeping counts for the gene panel of 63 only
[2022-07-19 14:20:52] INFO    (datatypes.py:385)  Single cell data: Grouping gene counts by cell type. Aggregating function is the mean.
[2022-07-19 14:20:52] INFO    (datatypes.py:387)  Grouped single cell data have 63 genes and 12 cell types
[2022-07-19 14:20:52] INFO    (app.py: 96)  Start cell typing
[2022-07-19 14:20:54] INFO    (ma